In [1]:
#!pip install pathos
#!pip install pymoo
#!pip install pymop

In [2]:
# Misc
import numpy as np
import time
import pandas as pd
import os,sys

# For PEARL
from neorl import PPO2,ACKTR,A2C
from neorl import MlpPolicy

print(os.path.dirname(os.path.abspath('../')),sys.path)
sys.path.append(os.path.dirname(os.path.abspath('../')))



	    NEORL: NeuroEvolution Optimisation with Reinforcement Learning
			 ███╗   ██╗███████╗ ██████╗ ██████╗ ██╗     
			 ████╗  ██║██╔════╝██╔═══██╗██╔══██╗██║     
			 ██╔██╗ ██║█████╗  ██║   ██║██████╔╝██║     
			 ██║╚██╗██║██╔══╝  ██║   ██║██╔══██╗██║     
			 ██║ ╚████║███████╗╚██████╔╝██║  ██║███████╗
			 ╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═╝  ╚═╝╚══════╝                                                
Copyright © 2021 Exelon Corporation (https://www.exeloncorp.com/) in collaboration with 
             MIT Nuclear Science and Engineering (https://web.mit.edu/nse/)
                             All Rights Reserved

                       

c:\Users\SEURPR\OneDrive - Idaho National Laboratory\Desktop\LDRD\LDRD microreactor performance\For_release\PEARL-MOOv1\PEARL-MOO ['c:\\Users\\SEURPR\\OneDrive - Idaho National Laboratory\\Desktop\\LDRD\\LDRD microreactor performance\\For_release\\PEARL-MOOv1\\PEARL-MOO\\examples\\unconstrained_optimization', 'c:\\Users\\SEURPR\\AppData\\Local\\anacond

In [3]:
from pearlmoo.methods.pearl import CreateEnvironment

In [4]:
from pearlmoo.utils.tools import uniform_reference_points # used to generate approprietaly spaced pareto front
from pearlmoo.utils.tools import isDominated
from pearlmoo.utils.hyper_volume import hypervolume
from pearlmoo.callbacks.pearlmoocalls import PEARLRLLogger

In [5]:
# Quickstart
# ---
from math import factorial
# Generate a multi-objective optimization problem from pymop
from pymop.factory import get_problem

PROBLEM = 'dtlz2'
NOBJ = 3
K =  5
NDIM = NOBJ + K - 1
P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
MU = int(H + (4 - H % 4))
BOUND_LOW, BOUND_UP = 0.0, 1.0
problem = get_problem(PROBLEM, n_var=NDIM, n_obj=NOBJ)
fitness = problem.evaluate# objective function
ref_points = uniform_reference_points(nobj = NOBJ, p = 12)    
pf = problem.pareto_front(ref_points)#get pareto front
#Setup the parameter space (d=5)
nx=NDIM
BOUNDS={}
for i in range(1,nx+1):
    BOUNDS['x'+str(i)]=['float', BOUND_LOW, BOUND_UP]
    
# Create PEARL environment
ncores = 8
env=CreateEnvironment(method='ppo', fit=fitness,nadir=[3,3,3],cl_penalty=64,buffer_size = 64,
    bounds=BOUNDS, mode='min',paradigm="unconstrained",ncores=ncores,selection='crowding',sorting="log",ref_points = None)
# Instantiate RL agents, use default hyper-parameter for now
ppo = PPO2(MlpPolicy, env=env, n_steps=32, ent_coef= 0.0001,gamma=0.99, learning_rate=0.0025, vf_coef=0.5, max_grad_norm=0.5, lam=0.95, nminibatches=4, noptepochs=10, cliprange=0.2)                       
#create a callback function to log data
cb=PEARLRLLogger(check_freq=5,plot_freq=1245,verbose=True)
# Perform the optimization
ppo.learn(total_timesteps=10000, callback=cb)
                             

Adding SHARED layer number -- 0 -- with  {'vf': [64, 64], 'pi': [64, 64]} nodes
Adding NON-SHARED layer number -- 0 -- with  {'vf': [64, 64], 'pi': [64, 64]} nodes
Adding NON-SHARED layer number -- 1 -- with  {'vf': [64, 64], 'pi': [64, 64]} nodes
Adding SHARED layer number -- 0 -- with  {'vf': [64, 64], 'pi': [64, 64]} nodes
Adding NON-SHARED layer number -- 0 -- with  {'vf': [64, 64], 'pi': [64, 64]} nodes
Adding NON-SHARED layer number -- 1 -- with  {'vf': [64, 64], 'pi': [64, 64]} nodes
----------------------------------------------------------------------------------
RL callback at step 40/10000
----------------------------------------------------------------------------------
----------------------------------------------------------------------------------
RL callback at step 80/10000
----------------------------------------------------------------------------------
----------------------------------------------------------------------------------
RL callback at step 120/10000
-

In [9]:

W,Y,Z,X,output = [],[],[],[],[]
buffer_pop = {}
for data in cb.pearl_hist['global_fitness']:
    buffer_pop[len(buffer_pop) + 1] = data
    X.append(data)
            
pearl_sol = []
for elem in [-np.array(x) for x in buffer_pop.values()]:
    isdominated = False
    for elem2 in pf:# Compare the elements generated to a known pareto front
        if isDominated(-np.array(elem),-elem2):
            isdominated = True
    if not isdominated:
        pearl_sol.append(elem)
    W.append(elem[0])
    Y.append(elem[1])
    Z.append(elem[2])


print("--- :")
print("Pareto Optimal solutions:",len(pearl_sol))
pearl_sol = np.unique(pearl_sol,axis=0)
print('Unique Pareto Optimal solutions',pearl_sol.shape)

if buffer_pop != {}:
    nadir = np.max([[-np.array(x) for x in buffer_pop.values()]],axis=1)[0]
    print("nadir",nadir)
    nadir_ref = [3,3,3]
    print('hyper volume PEARL ',hypervolume([tuple(x) for x in [-np.array(x) for x in buffer_pop.values()]], np.array(nadir_ref)))

# Plot the final pareto front versus the pre-generated one
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(W,Y,Z, marker = '*',color = 'red')
ax.scatter(np.array(pf)[:,0],np.array(pf)[:,1],np.array(pf)[:,2], marker = 'x',color = 'black')
ax.view_init(elev=30, azim=45)
ax.set_xlabel('f1')
ax.set_ylabel('f2')
ax.set_zlabel('f3')
plt.savefig('./'+PROBLEM+'.png')
plt.tight_layout()

--- :
Pareto Optimal solutions: 230
Unique Pareto Optimal solutions (230, 3)
nadir [1.07683156 1.04991486 1.15823042]
hyper volume PEARL  26.388367956326736
